In [1]:
import dash
#from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output,State
import base64
import datetime
import io
import dash_table
import pandas as pd
import dash_table
from PyPDF2 import PdfFileReader

In [ ]:
import re
def preprocess_resume(text_to_clean: str) -> str:
    """
    Does initial processing to the resume text before passing it to the model
    
    :param str text_to_clean: The text contained in the resume
    :return: processed text
    :rtype: str
    """
    # Replace all newlines and blanklines with special strings
    regx = re.compile(r"\n\n")
    text_to_clean = regx.sub(repl=" blankline ", string=text_to_clean)

    regx = re.compile(r"\n")
    text_to_clean = regx.sub(repl=" newline ", string=text_to_clean)


    # Make all white space a single space
    regx = re.compile(r"\s+")
    text_to_clean = regx.sub(repl=" ", string=text_to_clean)

    # Remove any trailing or leading white space
    text_to_clean = text_to_clean.strip(" ")
    return text_to_clean

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

colors = {
    'background': '#FAFDFF'
}


app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1("My Resume Score",style={'textAlign': 'left', 'color': '#1285F9'}),
    html.H3("Upload your resume",style={'textAlign': 'center', 'color': '#1C2833'}),
    dcc.Dropdown(
    options=[
        {'label': 'Engineering', 'value': 'Engineering'},
        {'label': 'Technology', 'value': 'Tech'},
        {'label': 'Accounting&Finance', 'value': 'Accounting & Finance'},
        {'label': 'Education', 'value': 'Education'}
    ],
    placeholder="Select your feild of study",
)  ,
    
    
    dcc.Dropdown(
        id='demo-dropdown',
    options=[
        {'label': 'Employed', 'value': 'Employed'},
        {'label': 'Looking for a job', 'value': 'looking'},
       
    ],
    placeholder="Select your employment status",
) ,
dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload'),
])


def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif 'pdf' in filename:
            # Assume that the user uploaded a pdf file
            pdf = PdfFileReader(io.BytesIO(decoded))
            data=""
            for page in pdf.pages:
                data += page.extractText()
            print(f'Before:\n\n{data}')
            data = preprocess_resume(data)
            print(f'After:)
            return html.Div([
                f'You uploaded a pdf file {data}'
            ])
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            data=df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])


@app.callback(Output('output-data-upload', 'children'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename'),
               State('upload-data', 'last_modified')])
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children

def update_output(value):
    return 'You have selected "{}"'.format(value)



#if __name__ == '__main__':
    app.run_server(debug=True)


if __name__ == '__main__':
#     model = joblib.load("tokenizer.pickle")
# Update model
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
